In [10]:
import json
from typing import List, Union
import itertools

import numpy as np
from collections import defaultdict


In [11]:
def estimate_pass_at_k(
    num_samples: Union[int, List[int], np.ndarray],
    num_correct: Union[List[int], np.ndarray],
    k: int
) -> np.ndarray:
    """
    Estimates pass@k of each problem and returns them in an array.
    """

    def estimator(n: int, c: int, k: int) -> float:
        """
        Calculates 1 - comb(n - c, k) / comb(n, k).
        """
        if n - c < k:
            return 1.0
        return 1.0 - np.prod(1.0 - k / np.arange(n - c + 1, n + 1))

    if isinstance(num_samples, int):
        num_samples_it = itertools.repeat(num_samples, len(num_correct))
    else:
        assert len(num_samples) == len(num_correct)
        num_samples_it = iter(num_samples)

    return np.array([estimator(int(n), int(c), k) for n, c in zip(num_samples_it, num_correct)])


In [17]:
names = ["Text_DaVinci_Raw_Output","Text_DaVinci_Refined_Output","GPT3.5_Raw_Output","GPT3.5_Refined_Output", "T5_Raw_Output","T5_Refined_Output"]
#names = ["Phi_Raw_Output","Phi_Refined_Output"]

matrix = "Compiled"
for name in names:
    print(name)
    filename = f'../Generation/{name}.json'
    outfilename = f'./{name}_{matrix}_Result.json'

    with open(outfilename, 'r') as f:
        result = json.load(f)

    results = defaultdict(list)

    for r in result:
        results[r['id']].append(r)

    # Calculate pass@k.
    total, correct = [], []
    for result in results.values():
        passed = [r["passed"] for r in result]
        total.append(len(passed))
        correct.append(sum(passed))
    total = np.array(total)
    correct = np.array(correct)
    # print(total, correct)
    ks = [1,3,10]
    pass_at_k = {f"pass@{k}": (estimate_pass_at_k(total, correct, k).mean())*100
                        for k in ks if (total >= k).all()}
    print(pass_at_k)


Text_DaVinci_Raw_Output
{'pass@1': 17.57217847769029, 'pass@3': 26.675415573053368, 'pass@10': 35.43307086614173}
Text_DaVinci_Refined_Output
{'pass@1': 26.968503937007878, 'pass@3': 37.3611111111111, 'pass@10': 46.3254593175853}
GPT3.5_Raw_Output
{'pass@1': 33.41207349081365, 'pass@3': 44.496937882764655, 'pass@10': 52.493438320209975}
GPT3.5_Refined_Output
{'pass@1': 33.55643044619423, 'pass@3': 44.37226596675415, 'pass@10': 52.887139107611546}
T5_Raw_Output
{'pass@1': 0.10498687664041995, 'pass@3': 0.13123359580052493, 'pass@10': 0.13123359580052493}
T5_Refined_Output
{'pass@1': 0.03937007874015749, 'pass@3': 0.10936132983377078, 'pass@10': 0.26246719160104987}


In [13]:
# Write a code for pass@k
def pass_at_k(results, ks):
    total = len(results)
    for k in ks:
        correct = 0
        for result in results.values():
            passed = [r["passed"] for r in result[:k]]
            correct += (1 if sum(passed)!=0 else 0)
        print(f"DFA-EQ@{k}: {(correct/total)*100}")

        

    

In [15]:
names = ["Text_DaVinci_Raw_Output","Text_DaVinci_Refined_Output","GPT3.5_Raw_Output","GPT3.5_Refined_Output", "T5_Raw_Output","T5_Refined_Output"]
names = ["Phi_Raw_Output","Phi_Refined_Output"]
matrix = "DFA"
for name in names:
    print(name)
    filename = f'../Generation/{name}.json'
    outfilename = f'./{name}_{matrix}_Result.json'

    with open(outfilename, 'r') as f:
        result = json.load(f)

    results = defaultdict(list)


    for r in result:
        results[r['id']].append(r)

    pass_at_k(results, [1,3,10])

    # Calculate pass@k.
    total, correct = [], []
    for result in results.values():
        passed = [r["passed"] for r in result]
        total.append(len(passed))
        correct.append(sum(passed))
    total = np.array(total)
    correct = np.array(correct)
    # print(total, correct)
    ks = [1,3,10]
    pass_at_k_result = {f"pass@{k}": (estimate_pass_at_k(total, correct, k).mean())*100
                        for k in ks if (total >= k).all()}
    print(pass_at_k_result)


Phi_Raw_Output
DFA-EQ@1: 1.4435695538057742
DFA-EQ@3: 2.3622047244094486
DFA-EQ@10: 6.299212598425196
{'pass@1': 1.0629921259842519, 'pass@3': 2.6399825021872267, 'pass@10': 6.299212598425196}
Phi_Refined_Output
DFA-EQ@1: 0.0
DFA-EQ@3: 2.3622047244094486
DFA-EQ@10: 5.6430446194225725
{'pass@1': 0.8792650918635171, 'pass@3': 2.3108048993875765, 'pass@10': 5.6430446194225725}
